<a href="https://colab.research.google.com/github/jamespan1118/Xenopus_signal_processing/blob/main/File_Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####Creating the folder structures from raw data

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import librosa
from librosa import display
import pandas as pd
import numpy as np
import os
import shutil
from pydub import AudioSegment 
from pydub.utils import make_chunks 
from scipy.io.wavfile import write
import warnings

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
worksheet = gc.open('Male Female Assay Log').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

import pandas as pd
df = pd.DataFrame.from_records(rows)
df.columns = df.iloc[0] 
df = df[1:]
df.head()

,Date,Separation Status,Recording_Tank,Hydraphone,Folder Name,Tank_Number_male,RFID_male,Species_male,Mass_male,hCG_injected_male,Tank_Number_Female,RFID_Female,Species_Female,Mass_Female,hCG_injected_Female,Comment,Frog Camera
1,06/14/22,Separate,1,1,20220614_F827E-F829C_F8340-F8297_F835F-F82C0,,F829C,laevis,50.1,0.2505,31,F837E,laevis,130.3,0.6515,"only recorded for 55 minutes, terminated unexp...",
2,06/14/22,Separate,2,2,20220614_F827E-F829C_F8340-F8297_F835F-F82C0,,F8297,laevis,46,0.23,31,F8340,laevis,98.7,0.4935,"only recorded for 55 minutes, terminated unexp...",
3,06/14/22,Separate,3,3,20220614_F827E-F829C_F8340-F8297_F835F-F82C0,,F82C0,laevis,56.3,0.2815,31,F835F,laevis,102.7,0.5135,"only recorded for 55 minutes, terminated unexp...",
4,06/15/22,Together,1,1,20220615_F827E-F829C_F8340-F8297_F835F-F82C0,,F829C,laevis,50.1,0.2505,31,F837E,laevis,130.3,0.6515,ticking found,
5,06/15/22,Together,2,2,20220615_F827E-F829C_F8340-F8297_F835F-F82C0,,F8297,laevis,46,0.23,31,F8340,laevis,98.7,0.4935,no ticking,


In [ ]:
overall_dir = os.path.dirname(os.path.realpath('/content/drive/MyDrive/Xenopus_male_female_data/overall/Male Female Assay Log.gsheet'))
raw_data_dir = '/content/drive/MyDrive/Xenopus_male_female_data/2022_phonotaxis_natural'

In [ ]:
def parse_folder_name(folder_name):
  pair_number = folder_name.count('_')
  pair_names = []
  date = folder_name.split('_')[0]
  for i in range(0,pair_number):
    pair_name = folder_name.split('_')[i+1]
    pair_names.append(pair_name)
  #search for species
  species_list = []
  for i in pair_names:
    female_ID = i.split('-')[1]
    #species = np.array(df.loc[df['RFID_Female']==female_ID]['Species_Female'])[0]
    species = df.loc[df['RFID_Female']==female_ID]['Species_Female'].values[0]
    species_list.append(species)
  assert(len(pair_names)==len(species_list))
  return pair_number, pair_names, species_list

In [ ]:
number,names,species_list = parse_folder_name('20220802_AD82B-AD7E3_AD804-AD827')
print(number)
print(len(names))
print(species_list)

2
2
['malawi', 'malawi']


In [ ]:
def make_pair_dir(folder_name,overall_dir):
  pair_number, pair_names, species_list = parse_folder_name(folder_name)
  for i in range(0,pair_number):
    current_species = species_list[i]
    species_dir = os.path.join(overall_dir,current_species)
    pair_name = pair_names[i]
    pair_dir = os.path.join(species_dir,pair_name)
    if not os.path.exists(pair_dir):
      os.mkdir(pair_dir)

In [ ]:
make_pair_dir('20220802_AD82B-AD7E3_AD804-AD827',overall_dir)

In [ ]:
#create txt files containing links in individual's pair dir
#folder_name is the folder containing the original raw data
def create_txt_file(folder_name,df):
  folder_dir = os.path.join(raw_data_dir,folder_name)
  #loop through all audio files
  for filename in os.listdir(folder_dir):
    if filename.endswith(".wav"):
      pair_ID = filename.split('-')[1]+'-'+filename.split('-')[2]
      female_ID = filename.split('-')[2]
      try:
        species = df.loc[df['RFID_Female']==female_ID]['Species_Female'].values[0]
        species_dir = os.path.join(overall_dir,species)
        pair_dir = os.path.join(species_dir,pair_ID)
        assert(os.path.exists(pair_dir))
        #put the link to the audio file in a text file in the pair_dir
        os.chdir(pair_dir)
        #first check whether rapping/ticking
        status = df.loc[df['RFID_Female']==female_ID]['Separation Status'].values[0]
        if(status=='Separate'):
          file_rapping = open(pair_ID+"_rapping.txt","a")
          full_path = os.path.join(raw_data_dir,folder_name,filename)
          assert(os.path.exists(full_path))
          file_rapping.write(full_path+'\n')
          file_rapping.close()
        else:
          file_ticking = open(pair_ID+"_ticking.txt","a")
          full_path = os.path.join(raw_data_dir,folder_name,filename)
          assert(os.path.exists(full_path))
          file_ticking.write(full_path+'\n')
          file_ticking.close()
      except(IndexError):
        pass
      except Exception as e:
          print('Failed to process %s. Reason: %s' % (filename, e))

In [ ]:
create_txt_file('20220802_AD82B-AD7E3_AD804-AD827',df)

####Iterate through the audio files to produce 1 hour chunks

In [ ]:
def chunking_pair(pair_dir):
  duets_dir = os.path.join(pair_dir,"duets")
  os.makedirs(duets_dir, exist_ok=True)

  chunk_dir = os.path.join(duets_dir,"chunks")
  os.makedirs(chunk_dir, exist_ok=True)

  try:
    rapping_dir = os.path.join(chunk_dir,"rapping")
    rapping_file_name = os.path.basename(pair_dir)+'_rapping.txt'
    rapping_file_path = os.path.join(pair_dir,rapping_file_name)
    read_txt_and_chunk(rapping_file_path,rapping_dir)
  except Exception as e:
    print('Rapping chunking fails')
    print(e)

  try:
    ticking_dir = os.path.join(chunk_dir,"ticking")
    ticking_file_name = os.path.basename(pair_dir)+'_ticking.txt'
    ticking_file_path = os.path.join(pair_dir,ticking_file_name)
    read_txt_and_chunk(ticking_file_path,ticking_dir)
  except Exception as e:
    print('Ticking chunking fails')
    print(e)

def read_txt_and_chunk(txt_file_path,output_path):
  file1 = open(txt_file_path, 'r')
  Lines = file1.readlines()
  list_line=[]
  for line in Lines:
    list_line.append(line[:-1])
  for i in list_line:
    chunking_file(i,output_path)

def chunking_file(file_path,output_path):
  file_name = os.path.basename(file_path)
  file_name = os.path.splitext(file_name)[0]
  '''
  myaudio = AudioSegment.from_file(file_path, "wav") 
  chunk_length_ms = 1000*60*60 # pydub calculates in millisec 
  chunks = make_chunks(myaudio,chunk_length_ms) #Make chunks of one hour
  for i, chunk in enumerate(chunks): 
      chunk_name = file_name+"{0}.wav".format(i) 
      print ("exporting", chunk_name)
      out = os.path.join(output_path,chunk_name)
      chunk.export(out, format="wav")
  '''
  i=0
  while(1):
    samples,sr=librosa.load(file_path,offset=i,duration=3600)
    if (len(samples)==0):
      break
    i=i+3600 #jump to the next hour
    chunk_name = file_name+"{0}.wav".format(i%3600+1) 
    out = os.path.join(output_path,chunk_name)
    librosa.output.write_wav(out, samples, sr)
    print ("exporting", chunk_name)

In [ ]:
file_path = '/content/drive/MyDrive/Xenopus_male_female_data/2022_phonotaxis_natural/20220802_AD82B-AD7E3_AD804-AD827/03-AD804-AD827-220802_1902.wav'
output_path = '/content/drive/MyDrive/Xenopus_male_female_data/overall/malawi/AD804-AD827/duets/chunks'
assert(os.path.exists(file_path))
assert(os.path.exists(output_path))
#chunking_file(file_path,output_path)

In [ ]:
file_name = os.path.basename(file_path)
file_name = os.path.splitext(file_name)[0]
i=0
duration = 1200

warnings.filterwarnings("ignore")
while(1):
  try:
    samples,sr=librosa.load(file_path,sr=44100,offset=i,duration=duration)
    chunk_name = file_name+"_"+str(i%duration+1) 
    out = os.path.join(output_path,chunk_name)
    #librosa.output.write_wav(out, samples, sr)
    write(out,sr,samples)
    print ("exporting", chunk_name)
    i=i+duration #jump to the next hour
  except:
    break

exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-220802_1902_1
exporting 03-AD804-AD827-

In [ ]:
!cd /content/drive/MyDrive/Xenopus_male_female_data/overall/malawi/AD804-AD827/duets/chunks

In [ ]:
!ls

AD82B-AD7E3_rapping.txt


####Clear out functions

In [ ]:
def clear_dir(folder):
  for filename in os.listdir(folder):
      file_path = os.path.join(folder, filename)
      try:
          if os.path.isfile(file_path) or os.path.islink(file_path):
              os.unlink(file_path)
          elif os.path.isdir(file_path):
              shutil.rmtree(file_path)
      except Exception as e:
          print('Failed to delete %s. Reason: %s' % (file_path, e))

def clear_four_species_dir():
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/laevis')
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/petersii')
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/poweri')
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/victorianus')
  clear_dir('/content/drive/MyDrive/Xenopus_male_female_data/overall/malawi')

In [ ]:
clear_four_species_dir()